In [1]:
import pyesapi
import atexit
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose);

In [2]:
patients_checked = []
structures = {}

In [3]:
i = 0
for pat_sum in app.PatientSummaries:
    print(i)
    i += 1
    if i > 200:
        break
    MRN = pat_sum.Id
    if MRN in patients_checked:
        continue
    patients_checked.append(MRN)
    app.ClosePatient()
    patient = app.OpenPatientById(MRN)
    courses = patient.Courses
    for course in courses:
        if course.Id.find('QA') == -1:
            plans = [i for i in course.PlanSetups]
            for plan in plans:
                if plan.StructureSet is None: # No structures present, move on
                    continue
                for structure in plan.StructureSet.Structures:
                    structure_name = structure.Id
                    if structure_name not in structures:
                        structures[structure_name] = {'PatientID': [], 'Course': [], 'Plan': []}
                    structures[structure_name]['PatientID'].append(MRN)
                    structures[structure_name]['Course'].append(course.Id)
                    structures[structure_name]['Plan'].append(plan.Id)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [7]:
list(structures.keys())

['BODY',
 'carina',
 'Cord',
 'GTV',
 'PTV',
 'Lung RT',
 'Lung LT',
 'Lung BILAT',
 'Heart',
 'itv',
 'Great Vessels',
 'Trach/ProxBronch',
 'Esophagus',
 'Ribs',
 'Chest wall',
 "ct sim bb's",
 'Pacemaker',
 'CouchSurface',
 'CouchInterior',
 '0 opti ptv',
 'Dose 1440[cGy]rp',
 'Dose 2400[cGy]rp',
 'Dose 30[%]ro',
 'Dose 50[%]ro',
 'Skin',
 'Diaphraghm Dome',
 '0 opti ring 50%',
 '0 ring 30%',
 'ctv',
 'ptv50',
 'gtv',
 'body',
 'body1',
 'cord',
 'ptv66',
 'c6',
 'block',
 'rt post neck',
 'ptv',
 'lung_lt',
 'l eye',
 'wire',
 'r eye',
 'r lens',
 'l lens',
 'Bolus 1CM',
 'lung_rt',
 'CT iso',
 'Spinal Canal',
 'Kidney LT',
 'Kidney RT',
 'Kidney Bilat',
 'GTV-sacrum-NLP',
 'GTV-L1-NLP',
 'Rt Kidney',
 'Rt Femoral Head',
 'ISO RT RIB',
 'ISO R HIP',
 'rt rib bb',
 'gtv rt rib',
 'gtv right hip',
 'T 11',
 'T7',
 'Dose 3600[cGy]',
 't1',
 'Dose 50[%] stern',
 'gtv1',
 'Isocenter Marker',
 'PTV1',
 'cord_E',
 'fiducials',
 'ring',
 'ring2',
 'tmp',
 'VB+1',
 'lungs',
 'lung-1',
 'VB'

In [8]:
structures['Brainstem']

{'PatientID': ['21201462', '21201462', '21201462'],
 'Course': ['IMSRT-Pitutary', 'IMSRT-Pitutary', 'IMSRT-Pitutary'],
 'Plan': ['7FIMSRT', '7FIMSRT1', '7FIMSRT#']}

In [ ]:
patient.CoursesLot('C1').PlanSetupsLot(0)